<a href="https://colab.research.google.com/github/merucode/Stock/blob/11_stock_info/22_theme_report(%EC%A0%80%EC%B6%9C%EC%82%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0. Install Dependency

In [ ]:
!pip install pykrx
!pip install dart-fss

# STEP 1. Infomation about Issue

### Google 검색어 트랜드
* [Google 검색어 트랜드](https://trends.google.co.kr/trends/explore?date=today%205-y&geo=KR&q=%EC%A0%80%EC%B6%9C%EC%82%B0&hl=ko)

### Issue
* 통계청 발표 이력
    * 매달 마지막 주 수요일에 2달 전 출생통계 발표
    * 2023-02-22 2022년 출생·사망통계(잠정)
    * 2023-08-30 2023년 6월 및 2분기 인구동향(출생, 사망, 혼인, 이혼)
* 통계청 발표 예정
    * 2023-11-29 2023년 10월 및 3분기 인구동향(출생, 사망, 혼인, 이혼)
    * 2024-02-29 2023년 잠정출산율 발표

### Reference Site
* [통계청 출생통계 보도자료](https://kostat.go.kr/board.es?mid=a10301010000&bid=a103010100)

# STEP 2. 종합 분석 Plot

### Step 2-1. Def

In [ ]:
from pykrx import stock           # 데이터 : 주식 데이터 전송
import datetime                   # 그래프 : 문자열 시간 전환
import matplotlib.pyplot as plt   # 그래프
%matplotlib inline

tickers_KOSDAQ = stock.get_market_ticker_list(market="KOSDAQ")
tickers_KOSPI = stock.get_market_ticker_list(market="KOSPI")
tickers = tickers_KOSDAQ + tickers_KOSPI
ticker_dict = dict()

for ticker in tickers:
    stock_name = stock.get_market_ticker_name(ticker)
    ticker_dict[stock_name] = ticker

def convert_ohlcv_format(df):
    df = df.reset_index()
    df = df[['날짜', '시가', '고가', '저가', '종가', '거래량']]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    return df

def plot_with_issue_date(start_date, end_date, stock_name_lst, ticker_dict, issue_date_lst,  year_lst):
    ### Get ohlcv
    ohlcv_kosdaq = stock.get_index_ohlcv(start_date, end_date, "2001")
    ohlcv_kosdaq = convert_ohlcv_format(ohlcv_kosdaq)

    ohlcv_lst = []
    for stock_name in stock_name_lst:
        ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker_dict[stock_name])
        ohlcv = convert_ohlcv_format(ohlcv)
        ohlcv['ad_close'] = ohlcv['close'] / ohlcv_kosdaq['close']
        ohlcv_lst.append(ohlcv)

    ### Convert str date to datetime date
    issue_date_dt_lst = []    # issue_date
    for issue_date_str in issue_date_lst:
        issue_date_dt_lst.append(datetime.datetime.strptime(issue_date_str, '%Y-%m-%d'))

    year_dt_lst = []    # year
    for year_str in year_lst:
        year_dt_lst.append(datetime.datetime.strptime(year_str, '%Y-%m-%d'))


    ### Plot
    fig, ax = plt.subplots(len(stock_name_lst)+1, 2, figsize=(14, 4+4*len(stock_name_lst)), constrained_layout=True)
    # figsize : 가로 10, 세로 stock_name_lst 갯수
    # constrained_layout=True : 서브플롯 간의 간격 자동 조절

    plot_color_lst = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

    for ohlcv, stock_name in zip(ohlcv_lst, stock_name_lst):     # close price line plot
        ax[0][0].plot(ohlcv['date'], ohlcv['close']/ohlcv['close'][0], label=stock_name)
        ax[0][0].set_title('종합 그래프')

        ax[0][1].plot(ohlcv['date'], ohlcv['ad_close']/ohlcv['ad_close'][0], label=stock_name)
        ax[0][1].set_title('종합 그래프 adjusted by KOSDAQ')

    for i, ohlcv in enumerate(ohlcv_lst):     # close price adjust by KOSDAQ line plot
        ax[i+1][0].plot(ohlcv['date'], ohlcv['close'], color=plot_color_lst[i])
        ax[i+1][0].set_title(f'{stock_name_lst[i]}')

        ax[i+1][1].plot(ohlcv['date'], ohlcv['ad_close'], color=plot_color_lst[i])
        ax[i+1][1].set_title(f'{stock_name_lst[i]} adjusted by KOSDAQ')


    for pt in issue_date_dt_lst:    # add issue date line
        for i in range(0, len(stock_name_lst)+1):
            ax[i][0].axvline(x=pt, color='#D62728')
            ax[i][1].axvline(x=pt, color='#D62728')

    for pt in year_dt_lst:    # add year line
        for i in range(0, len(stock_name_lst)+1):
            ax[i][0].axvline(x=pt, color='#EFE034')
            ax[i][1].axvline(x=pt, color='#EFE034')

    ax[0][0].legend()
    ax[0][1].legend()
    plt.show()

### Step 2-2. Plot : '크린앤사이언스', '하츠', '위니아', '위닉스', '웰크론'

In [ ]:
start_date = '2023-01-01'
end_date = '2023-10-29'
stock_name_lst = ['제로투세븐', '아가방컴퍼니', '메디앙스']
# 1st Apply : '제로투세븐', '아가방컴퍼니', '메디앙스'
# 2nd Apply : '유엔젤'(AI)
# Except : '삼성출판사'(반응X), '캐리소프트'(반응小), '토박스코리아'(반응小), 네오팜(반응X)
# Wait : '꿈비'(늦게상장)
year_lst = ['2023-01-01']
issue_date_lst = ['2023-02-22','2023-08-30']

plot_with_issue_date(start_date, end_date, stock_name_lst, ticker_dict, issue_date_lst, year_lst)

# STEP 3. 개별분석

### Step 3-1. Def

In [ ]:
### 연도별 그래프
def plot_per_year(start_date, end_date, stock_name, ticker_dict, std_month):
    ### Get ohlcv
    ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker_dict[stock_name])
    ohlcv = convert_ohlcv_format(ohlcv)
    ohlcv_kosdaq = stock.get_index_ohlcv(start_date, end_date, "2001")
    ohlcv_kosdaq = convert_ohlcv_format(ohlcv_kosdaq)
    ohlcv_kosdaq['close'] = ohlcv_kosdaq['close']
    ohlcv['ad_close'] = ohlcv['close'] / ohlcv_kosdaq['close']

    ### Plot
    y_lst = [2023]

    fig, ax = plt.subplots(2, 2, figsize=(14, 10), constrained_layout=True)
    for y in y_lst:
        filter = (f'{y+1}-{std_month}-01' > ohlcv['date']) & (ohlcv['date'] > f'{y}-{std_month}-01')
        ohlcv_y = ohlcv[filter]
        ohlcv_y = ohlcv_y.reset_index()
        ax[0][0].plot(ohlcv_y['close'], label=f'{str(y)}-{str(std_month)} ~ {str(y+1)}-{str(std_month-1)}')     # 종가
        ax[0][1].plot(ohlcv_y['ad_close'], label=f'{str(y)}-{str(std_month)} ~ {str(y+1)}-{str(std_month-1)}')  # 수정종가
        ax[1][0].plot(ohlcv_y['close']/ohlcv_y['close'][0], label=f'{str(y)}-{str(std_month)}  ~ {str(y+1)}-{str(std_month-1)}') # 시작점 대비 변화


    for pt in range(20, 260, 20):    # add month line
        ax[0][0].axvline(x=pt, color='#EFE034')
        ax[0][1].axvline(x=pt, color='#EFE034')
        ax[1][0].axvline(x=pt, color='#EFE034')


    ax[0][0].set_title('종가 연도별 그래프')
    ax[0][1].set_title('수정종가 연도별 그래프')
    ax[1][0].set_title('시작점 대비 변화 연도별 그래프')

    ax[0][0].legend()
    ax[0][1].legend()
    ax[1][0].legend()
    plt.show()

In [ ]:
def convert_cap_format(df):
    df = df.reset_index()
    df = df[['날짜', '시가총액', '상장주식수']]
    df.columns = ['date', 'cap', 'stocks']
    return df

### 상장주식 수
def plot_cap(start_date, end_date, stock_name, ticker_dict):
    ### Get cap data
    cap = stock.get_market_cap(start_date, end_date, ticker_dict[stock_name])
    cap = convert_cap_format(cap)

    fig, ax = plt.subplots(1, 2, figsize=(14, 5), constrained_layout=True)
    ax[0].plot(cap['date'], cap['cap'], color='C0', label='시가총액')
    ax2 = ax[0].twinx()
    ax2.plot(cap['date'], cap['stocks'], color='C1', label='상장주식수')

    ax[0].legend(loc='upper left')
    ax2.legend()

    ax[0].set_title('시가총액&상장주식수')
    ax[1].set_title('')
    plt.show()

In [ ]:
import dart_fss as dart

api_key='###'
dart.set_api_key(api_key=api_key)

### 제로투세븐

In [ ]:
start_date = '2023-01-01'
end_date = '2023-10-30'
stock_name = '제로투세븐'
std_month = 1

plot_per_year(start_date, end_date, stock_name, ticker_dict, std_month)

In [ ]:
plot_cap('2017-01-01', end_date, stock_name, ticker_dict)

In [ ]:
reports = dart.filings.search(corp_code=ticker_dict[stock_name], bgn_de='20170101', end_de='20231030', pblntf_detail_ty='b001')
reports

### 아가방컴퍼니

In [ ]:
start_date = '2023-01-01'
end_date = '2023-10-30'
stock_name = '아가방컴퍼니'
std_month = 1

plot_per_year(start_date, end_date, stock_name, ticker_dict, std_month)

In [ ]:
plot_cap('2017-01-01', end_date, stock_name, ticker_dict)

In [ ]:
reports = dart.filings.search(corp_code=ticker_dict[stock_name], bgn_de='20170101', end_de='20231030', pblntf_detail_ty='b001')
reports

### 메디앙스

In [ ]:
start_date = '2023-01-01'
end_date = '2023-10-30'
stock_name = '메디앙스'
std_month = 1

plot_per_year(start_date, end_date, stock_name, ticker_dict, std_month)

In [ ]:
plot_cap('2017-01-01', end_date, stock_name, ticker_dict)

In [ ]:
reports = dart.filings.search(corp_code=ticker_dict[stock_name], bgn_de='20170101', end_de='20231030', pblntf_detail_ty='b001')
reports

# STEP 4. Report


### Issue

### Related Stock
*

### Plan
* Enter
  * 10~12월
* Exit
  * 1~3월 : 50% 7500 1/2